In [6]:
import pandas as pd
import torch
from torch_geometric.data import Data
import ipaddress
from sklearn.preprocessing import MaxAbsScaler

def load_star_multigraph_from_csv(file_path):
    df = pd.read_csv(file_path)

    # Clean column names by stripping extra spaces
    df.columns = df.columns.str.strip()

    def is_valid_ipv4(ip):
        try:
            ipaddress.IPv4Address(ip)
            return True
        except ipaddress.AddressValueError:
            return False

    # Filter out invalid IP addresses
    valid_ip_mask = df.apply(
        lambda row: is_valid_ipv4(row["Source IP"]) and is_valid_ipv4(row["Destination IP"]),
        axis=1,
    )
    df = df[valid_ip_mask]

    # Drop rows with missing values in critical columns
    df = df.dropna(subset=["Source IP", "Destination IP", "Label"])

    def ip_address(ip):
        ip_integer = int(ipaddress.IPv4Address(ip))
        return ip_integer

    df["Source IP"] = df["Source IP"].apply(lambda i: ip_address(i))
    df["Destination IP"] = df["Destination IP"].apply(lambda i: ip_address(i))
    df = df.dropna()

    # Extract relevant columns
    source_nodes = df["Source IP"].astype("category").cat.codes
    target_nodes = df["Destination IP"].astype("category").cat.codes
    edge_index = torch.tensor([source_nodes.values, target_nodes.values], dtype=torch.long)

    # Tạo edge_id để phân biệt các cạnh song song
    df["edge_id"] = df.apply(
        lambda row: tuple(sorted((row["Source IP"], row["Destination IP"]))), axis=1
    )
    df["edge_id"] = df["edge_id"].astype("category").cat.codes

    edge_id = torch.tensor(df["edge_id"].values, dtype=torch.long)

    # Update feature columns to match those in the DataFrame
    feature_columns = [
        "Flow Duration", "Total Fwd Packets", "Total Backward Packets", 
        "Total Length of Fwd Packets", "Total Length of Bwd Packets", 
        "Fwd Packet Length Max", "Fwd Packet Length Min", "Fwd Packet Length Mean", 
        "Fwd Packet Length Std", "Bwd Packet Length Max", "Bwd Packet Length Min", 
        "Bwd Packet Length Mean", "Bwd Packet Length Std", "Flow Bytes/s", 
        "Flow Packets/s", "Flow IAT Mean", "Flow IAT Std", "Flow IAT Max", 
        "Flow IAT Min", "Fwd IAT Total", "Fwd IAT Mean", "Fwd IAT Std", 
        "Fwd IAT Max", "Fwd IAT Min", "Bwd IAT Total", "Bwd IAT Mean", 
        "Bwd IAT Std", "Bwd IAT Max", "Bwd IAT Min", "Fwd PSH Flags", 
        "Bwd PSH Flags", "Fwd URG Flags", "Bwd URG Flags", "Fwd Header Length", 
        "Bwd Header Length", "Fwd Packets/s", "Bwd Packets/s", "Min Packet Length", 
        "Max Packet Length", "Packet Length Mean", "Packet Length Std", 
        "Packet Length Variance", "FIN Flag Count", "SYN Flag Count", "RST Flag Count", 
        "PSH Flag Count", "ACK Flag Count", "URG Flag Count", "CWE Flag Count", 
        "ECE Flag Count", "Down/Up Ratio", "Average Packet Size", 
        "Avg Fwd Segment Size", "Avg Bwd Segment Size", "Fwd Header Length.1", 
        "Fwd Avg Bytes/Bulk", "Fwd Avg Packets/Bulk", "Fwd Avg Bulk Rate", 
        "Bwd Avg Bytes/Bulk", "Bwd Avg Packets/Bulk", "Bwd Avg Bulk Rate", 
        "Subflow Fwd Packets", "Subflow Fwd Bytes", "Subflow Bwd Packets", 
        "Subflow Bwd Bytes", "Init_Win_bytes_forward", "Init_Win_bytes_backward", 
        "act_data_pkt_fwd", "min_seg_size_forward", "Active Mean", "Active Std", 
        "Active Max", "Active Min", "Idle Mean", "Idle Std", "Idle Max", "Idle Min"
    ]

    # Ensure that all feature columns are present
    missing_features = [col for col in feature_columns if col not in df.columns]
    if missing_features:
        raise KeyError(f"Missing columns in DataFrame: {', '.join(missing_features)}")

    edge_attr = torch.tensor(df[feature_columns].fillna(0).values, dtype=torch.float)

    # Number of nodes
    num_nodes = max(source_nodes.max().item(), target_nodes.max().item()) + 1

    graph_data = data = Data(
        edge_index=edge_index,
        edge_attr=edge_attr,
        num_nodes=num_nodes,
        edge_id=edge_id,
    )

    return graph_data

In [8]:
import glob
directories = [
    "Star_data/Benign/*.csv",
]

file_count = {}
graph_data = []
idx = 1
for directory in directories:
    file_paths = glob.glob(directory)
    file_count[directory] = len(file_paths)

    for file_path in file_paths:
        graph_data = load_star_multigraph_from_csv(file_path)
        graph_data.y = 0
        torch.save(graph_data, f"Star_graph/Benign/Benign_{idx}.pt")
        print(idx)
        print(f"Number of nodes: {graph_data.num_nodes}")
        print(f"Number of edges: {graph_data.edge_index.size(1)}")
        print(f"Edge feature dimensions: {graph_data.edge_attr.size(1)}")
        print(f"Label: {graph_data.y}")
        idx = idx + 1

1
Number of nodes: 89
Number of edges: 679
Edge feature dimensions: 77
Label: 0
2
Number of nodes: 39
Number of edges: 348
Edge feature dimensions: 77
Label: 0
3
Number of nodes: 90
Number of edges: 901
Edge feature dimensions: 77
Label: 0
4
Number of nodes: 98
Number of edges: 601
Edge feature dimensions: 77
Label: 0
5
Number of nodes: 56
Number of edges: 460
Edge feature dimensions: 77
Label: 0
6
Number of nodes: 56
Number of edges: 448
Edge feature dimensions: 77
Label: 0
7
Number of nodes: 97
Number of edges: 1288
Edge feature dimensions: 77
Label: 0
8
Number of nodes: 147
Number of edges: 1057
Edge feature dimensions: 77
Label: 0
9
Number of nodes: 48
Number of edges: 480
Edge feature dimensions: 77
Label: 0
10
Number of nodes: 99
Number of edges: 1312
Edge feature dimensions: 77
Label: 0
11
Number of nodes: 59
Number of edges: 464
Edge feature dimensions: 77
Label: 0
12
Number of nodes: 69
Number of edges: 448
Edge feature dimensions: 77
Label: 0
13
Number of nodes: 107
Number of